# Generic optimization problem

This notebook is performing a 3 bit minimization for the following problem 

$$
x*y - x*z + 2*y*z + x - 2*y - 3*z
$$

## **Step 0**: Setup

In [1]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import RealAmplitudes

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# quadratic_program
from quadratic_program import QuadraticProgram

# Docplex - classical description of optimization problems
from docplex.mp.model import Model

# translations
from translators import docplex_mp_to_qp
from translators import qubo_to_sparse_pauli_op

# workflows 
from workflows import QuadraticProgramPostprocess, QuadraticProgramConverter

# SPSA
from spsa import minimize_spsa

## Load the IBM Quantum service (if using)

In [2]:
#service = QiskitRuntimeService()

In [3]:
#backend = service.get_backend('ibm_nazca')

## **Step 1** Map the problem to a Quantum Native format (Set of Operators, and a set of Quantum Circuits)

Specify optimization problem using docplex and convert to Quadratic Unconstained Binary Opimization (QUBO) problem that can be cast as an Ising Hamiltonian suitable for a quantum solution.

In [4]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
z = mdl.binary_var("z")
mdl.minimize(x*y - x*z + 2*y*z + x - 2*y - 3*z)
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x - 2 y - 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



### Convert to our `QuadraticProgram` format

In [5]:
qp = docplex_mp_to_qp(mdl)
print(qp.prettyprint())

Problem name: docplex model

Minimize
  x*y - x*z + 2*y*z + x - 2*y - 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



### Classical transformation to QUBO problem and Ising Hamiltonian

In [6]:
quadratic_transformer = QuadraticProgramConverter()
qubo = quadratic_transformer.run(qp)
print(qubo.prettyprint())

Problem name: docplex model

Minimize
  x*y - x*z + 2*y*z + x - 2*y - 3*z

Subject to
  No constraints

  Binary variables (3)
    x y z



In [7]:
hamiltonian, offset = qubo_to_sparse_pauli_op(qubo)
hamiltonian

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI'],
              coeffs=[-0.5 +0.j,  0.25+0.j,  1.25+0.j,  0.25+0.j, -0.25+0.j,  0.5 +0.j])

### Define ansatz circuit from circuit library

In [8]:
ansatz = RealAmplitudes(hamiltonian.num_qubits, entanglement = 'linear', reps=2)
print(ansatz.decompose())

     ┌──────────┐     ┌──────────┐                 ┌──────────┐            
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[3]) ├──────────────■──┤ Ry(θ[6]) ├────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ Ry(θ[1]) ├┤ X ├─────■──────┤ Ry(θ[4]) ├┤ X ├─────■──────┤ Ry(θ[7]) ├
     ├──────────┤└───┘   ┌─┴─┐    ├──────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ Ry(θ[2]) ├────────┤ X ├────┤ Ry(θ[5]) ├────────┤ X ├────┤ Ry(θ[8]) ├
     └──────────┘        └───┘    └──────────┘        └───┘    └──────────┘


## **Step 2**: Optimize the circuits and the operators to be measured

In [9]:
from qiskit.compiler import transpile
ansatz_ibm = transpile(ansatz, basis_gates = ['cz', 'sx', 'rz'],  coupling_map =[[0, 1], [1, 2]], optimization_level=3)
print(ansatz_ibm)

global phase: π
         ┌────┐┌──────────────┐   ┌────┐   ┌────────┐      ┌────┐  »
q_0 -> 0 ┤ √X ├┤ Rz(θ[0] + π) ├───┤ √X ├───┤ Rz(3π) ├─■────┤ √X ├──»
         ├────┤├──────────────┤┌──┴────┴──┐└─┬────┬─┘ │ ┌──┴────┴─┐»
q_1 -> 1 ┤ √X ├┤ Rz(θ[1] + π) ├┤ Rz(-π/2) ├──┤ √X ├───■─┤ Rz(π/2) ├»
         ├────┤├──────────────┤├──────────┤  ├────┤     └─────────┘»
q_2 -> 2 ┤ √X ├┤ Rz(θ[2] + π) ├┤ Rz(-π/2) ├──┤ √X ├────────────────»
         └────┘└──────────────┘└──────────┘  └────┘                »
«         ┌──────────────┐   ┌────┐  ┌────────┐                          »
«q_0 -> 0 ┤ Rz(θ[3] + π) ├───┤ √X ├──┤ Rz(3π) ├──────────────────────────»
«         └────┬────┬────┘┌──┴────┴─┐└────────┘  ┌────┐  ┌──────────────┐»
«q_1 -> 1 ─────┤ √X ├─────┤ Rz(π/2) ├────■───────┤ √X ├──┤ Rz(θ[4] + π) ├»
«              └────┘     └─────────┘    │     ┌─┴────┴─┐└────┬────┬────┘»
«q_2 -> 2 ───────────────────────────────■─────┤ Rz(-π) ├─────┤ √X ├─────»
«                                              └───

In [10]:
from permute_sparse_pauli_op import permute_sparse_pauli_op

layout = ansatz_ibm.layout.initial_layout
hamiltonian_ibm = permute_sparse_pauli_op(hamiltonian,layout, ansatz.qubits)
hamiltonian_ibm

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI'],
              coeffs=[-0.5 +0.j,  0.25+0.j,  1.25+0.j,  0.25+0.j, -0.25+0.j,  0.5 +0.j])

## **Step 3**: Execute using a quantum primitive function (estimator or sampler)

### Standard cost function definition

In [11]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [12]:
#session = Session(backend=backend)
#estimator = Estimator(session=session, options={"shots": int(1e4)})
#sampler = Sampler(session=session, options={"shots": int(1e4)})
estimator = QiskitEstimator(options={"shots": int(1e4)})
sampler = QiskitSampler(options={"shots": int(1e4)})

### Perform minimization using classical SPSA optimizer

In [13]:
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0, args=(ansatz_ibm, hamiltonian_ibm, estimator), maxiter=100)
res

 message: Optimization terminated successfully.
 success: True
     fun: -1.4986421285388796
       x: [ 6.575e+00  3.558e+00  1.494e+00  1.238e+00  3.147e+00
           -7.283e-02 -6.912e-02  3.080e+00  1.555e+00]
     nit: 100
    nfev: 200

### Computute sampled distribution at optimal values

In [14]:
# Assign solution parameters to ansatz
qc = ansatz_ibm.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
#session.close()

## **Step 4**: Post-processing of the results to return either a plot or the answer
Transform quantum solution and convert back into classical variable space

In [15]:
solution = QuadraticProgramPostprocess(qubo, quadratic_transformer).run(samp_dist)
solution

array([0., 0., 1.])